In [3]:
import csv
import pandas as pd
df = pd.read_csv('kerusakan_laptop.csv')

df

,Fan,Harddisk,Processor,Layar,Sistem Operasi,USB,Charging,Keyboard,RAM,Motherboard,Baterai,Bios,Touchpad,Audio,Jenis Kerusakan
0,2,3,1,4,2,3,1,4,2,3,1,4,2,2,Fan Tidak Berfungsi
1,1,1,2,3,1,4,4,3,1,4,4,3,2,4,Harddisk Rusak
2,3,4,4,2,3,2,3,4,4,3,4,2,4,3,Processor Overheating
3,4,2,1,4,2,3,4,1,3,4,3,2,3,1,Layar Blank
4,4,4,3,1,4,1,2,4,4,2,4,3,1,4,Sistem Operasi Error
5,2,3,2,3,1,4,4,3,2,1,2,4,4,3,Port USB Tidak Terbaca
6,1,4,4,2,3,4,1,4,3,3,1,4,2,4,Charging Tidak Berfungsi
7,3,2,1,4,2,2,4,4,1,4,4,1,3,2,Keyboard Rusak
8,4,3,2,4,4,3,3,1,4,2,4,2,4,3,RAM Tidak Terdeteksi
9,4,4,3,1,1,1,2,3,2,4,1,4,1,4,Motherboard Rusak


In [ ]:
import pandas as pd
import math
import csv

# Nama file CSV untuk menyimpan basis kasus
case_base_file = 'kerusakan_laptop.csv'

# Mapping untuk tingkat kerusakan
tingkat_kerusakan_mapping = {
    1: "Tidak ada masalah, Komponen dalam kondisi sempurna.",
    2: "Perlu Perawatan, dapat diperbaiki dengan servis ringan.",
    3: "Bermasalah, membutuhkan perbaikan besar.",
    4: "Rusak Total, Komponen tidak dapat digunakan dan harus diganti."
}

# Fungsi untuk membaca basis kasus dari file CSV
def load_case_base(file_path):
    case_base = []
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            case_base.append({
                "criteria": {
                    "Fan": int(row["Fan"]),
                    "Harddisk": int(row["Harddisk"]),
                    "Processor": int(row["Processor"]),
                    "Layar": int(row["Layar"]),
                    "Sistem Operasi": int(row["Sistem Operasi"]),
                    "USB": int(row["USB"]),
                    "Charging": int(row["Charging"]),
                    "Keyboard": int(row["Keyboard"]),
                    "RAM": int(row["RAM"]),
                    "Motherboard": int(row["Motherboard"]),
                    "Baterai": int(row["Baterai"]),
                    "Bios": int(row["Bios"]),
                    "Touchpad": int(row["Touchpad"]),
                    "Audio": int(row["Audio"])
                },
                "jenis_kerusakan": row["Jenis Kerusakan"]
            })
    return case_base

# Fungsi untuk mengonversi kriteria menjadi vektor numerik
def criteria_to_vector(criteria):
    return [
        criteria["Fan"],
        criteria["Harddisk"],
        criteria["Processor"],
        criteria["Layar"],
        criteria["Sistem Operasi"],
        criteria["USB"],
        criteria["Charging"],
        criteria["Keyboard"],
        criteria["RAM"],
        criteria["Motherboard"],
        criteria["Baterai"],
        criteria["Bios"],
        criteria["Touchpad"],
        criteria["Audio"]
    ]

# Fungsi untuk menghitung Cosine Similarity
def cosine_similarity(vector1, vector2):
    pembilang = sum(vec1 * vec2 for vec1, vec2 in zip(vector1, vector2))
    penyebut1 = math.sqrt(sum(vec1 ** 2 for vec1 in vector1))
    penyebut2 = math.sqrt(sum(vec2 ** 2 for vec2 in vector2))
    
    if penyebut1 == 0 or penyebut2 == 0:
        return 0
    
    return pembilang / (penyebut1 * penyebut2)

# Fungsi untuk mencari kasus yang paling mirip
def find_most_similar_case(case_base, new_criteria):
    most_similar_case = None
    max_similarity = 0
    similarity_results = []
    
    new_criteria_vector = criteria_to_vector(new_criteria)
    
    for i, case in enumerate(case_base, start=1):
        case_vector = criteria_to_vector(case["criteria"])
        similarity = cosine_similarity(new_criteria_vector, case_vector)
        
        similarity_results.append({
            "Case ID": i,
            "Criteria": case["criteria"],
            "Jenis Kerusakan": case["jenis_kerusakan"],
            "Similarity": similarity
        })
        
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_case = case
            
    return most_similar_case, similarity_results, max_similarity

# Fungsi untuk memberikan rekomendasi kerusakan
def recommend_cbr(case_base, new_criteria):
    similar_case, similarity_results, max_similarity = find_most_similar_case(case_base, new_criteria)
    
    df = pd.DataFrame(similarity_results)
    df = df.sort_values(by="Similarity", ascending=False).reset_index(drop=True)
    df.index += 1
    
    print("\nTabel Hasil Similaritas dan Ranking:")
    print(df[["Case ID", "Jenis Kerusakan", "Similarity"]])
    
    if max_similarity >= 0.8:  # Ambang batas similarity
        jenis_kerusakan = similar_case['jenis_kerusakan']
        print(f"\nKasus kerusakan yang paling mirip: {jenis_kerusakan} (Similarity: {max_similarity:.2f})")
    else:
        print("\nTidak ada kasus yang cukup mirip dengan ambang batas similarity.")
        jenis_kerusakan = "Tidak ditemukan rekomendasi."
    
    return jenis_kerusakan, max_similarity

def get_new_criteria():
    print("Masukkan nilai kriteria laptop baru (1-4):")
    criteria = {}
    for component in ["Fan", "Harddisk", "Processor", "Layar", "Sistem Operasi", "USB", 
                      "Charging", "Keyboard", "RAM", "Motherboard", "Baterai", "Bios", 
                      "Touchpad", "Audio"]:
        while True:
            try:
                # Mengambil input dan memeriksa apakah kosong
                value = input(f"{component}: ").strip()
                
                # Pastikan input tidak kosong
                if not value:
                    raise ValueError("Input tidak boleh kosong.")
                
                # Mengubah input menjadi integer
                value = int(value)
                
                # Validasi rentang nilai (1 hingga 4)
                if value < 1 or value > 4:
                    raise ValueError("Nilai harus antara 1 hingga 4.")
                
                # Menyimpan input yang valid
                criteria[component] = value
                break
            except ValueError as e:
                print(f"Input tidak valid: {e}. Silakan coba lagi.")
    return criteria

def save_new_case(new_criteria, jenis_kerusakan):
    # Menyimpan kasus baru ke dalam file CSV
    new_case = {**new_criteria, "Jenis Kerusakan": jenis_kerusakan}
    
    with open(case_base_file, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=new_case.keys())
        
        # Jika file kosong, tulis header terlebih dahulu
        if file.tell() == 0:
            writer.writeheader()
        
        writer.writerow(new_case)
        print("\nData kasus baru berhasil disimpan ke dalam CSV.")

# Fungsi untuk menampilkan hasil rekomendasi dengan format yang lebih baik
def display_recommendation(jenis_kerusakan, max_similarity):
    print("\n" + "="*50)
    print("                HASIL DIAGNOSA KERUSAKAN")
    print("="*50)
    
    if jenis_kerusakan == "Tidak ditemukan rekomendasi.":
        print(f"⚠️  Tidak ada kasus yang cukup mirip dengan ambang batas similarity.")
        print(f"    Silakan lakukan analisis lebih lanjut.")
    else:
        print(f"✅  Jenis Kerusakan yang Direkomendasikan:")
        print(f"    {jenis_kerusakan}")
        print(f"    (Tingkat Kemiripan: {max_similarity:.2%})")
    
    print("="*50)

# Main program
if __name__ == "__main__":
    # Load basis kasus dari CSV
    case_base = load_case_base(case_base_file)
    
    while True:
        # Dapatkan input kriteria baru
        new_criteria = get_new_criteria()
        
        # Rekomendasi jenis kerusakan
        jenis_kerusakan, max_similarity = recommend_cbr(case_base, new_criteria)
        
        # Menampilkan hasil diagnosis kerusakan
        display_recommendation(jenis_kerusakan, max_similarity)
        
        # Menyimpan kasus baru ke CSV
        save_new_case(new_criteria, jenis_kerusakan)
        
        # Menanyakan apakah pengguna ingin memasukkan data lagi
        another = input("\nIngin menganalisa kerusakan lainnya? (y/n): ").strip().lower()
        if another != 'y':
            print("\nCopyright Muhamad Nur Hanif")
            break

Masukkan nilai kriteria laptop baru (1-4):

Tabel Hasil Similaritas dan Ranking:
    Case ID           Jenis Kerusakan  Similarity
1        12              BIOS Corrupt    0.907993
2        16     Overheating Processor    0.881978
3         2            Harddisk Rusak    0.873228
4        14        Audio Tidak Keluar    0.870978
5         3     Processor Overheating    0.869130
6         9      RAM Tidak Terdeteksi    0.858880
7        20            Mesin Overload    0.856262
8         4               Layar Blank    0.850215
9         5      Sistem Operasi Error    0.848057
10       17               Layar Retak    0.844737
11       11     Baterai Tidak Mengisi    0.830713
12        8            Keyboard Rusak    0.830336
13       15     Harddisk Not Detected    0.810889
14        6    Port USB Tidak Terbaca    0.801756
15       18  Port USB Tidak Berfungsi    0.772825
16       13  Touchpad Tidak Berfungsi    0.771517
17       10         Motherboard Rusak    0.749151
18       21     Pro